In [2]:
import polars as pl
# from utils.tfrecords import pl2tf

In [3]:
fd=pl.read_csv_batched("./part-00000-f8f702fe-fc48-48c9-9326-0b7ca76ecd77-c000.csv",batch_size=3024000)

In [4]:
df=fd.next_batches(1)[0]

In [15]:
df["pay_flag"].sum()/df["finalClickFlag"].sum()

0.000783927970314213

In [14]:
df["finalClickFlag"].sum()/df["finalClickFlag"].shape[0]

0.4931805680669001

In [7]:
viptypes_map={
    "[]":0,
    "[MOVIES]":1,
    "[KIDS]":2,
    '[MOVIES, KIDS]':3
}
asrgender_map={
    "unknown":0,
    "child":1,
    "female":2,
    "male":3
}
respaytype_map={
    "FREE":1,
    "PAY":2
}
paytypematchflag_map={
    False:0,
    True:1
}
selfFlag_map={
    False:0,
    True:1
}
category_map={
    "儿童":1,
    "电影":2,
    "电视剧":3,
    "动漫":4,
    "音乐":5,
    "综艺":6,
    "电子竞技":7,
    "教育":8,
    "纪录片":9
}
viptypeuserpaycnt_split=[0,1,2,3]
viptypeduetime_split=[-180,-90,-30,-14,-7,-3,1,7,14,30,90,180]
# categoryusercnt_split=[0,2,5,10,17,27,43,74]
# ipresusercnt=[0, 1, 2, 3, 4, 6, 9, 12, 17, 24, 39]
def interval2id(col,split):
    evalstr="pl"
    for i in range(len(split)):
        evalstr+=f".when(pl.col('{col}')<={split[i]}).then(pl.lit({i}))"
    evalstr+=f".otherwise({i+1})"
    return eval(evalstr)


## 分箱

In [8]:
split_tab={}
breakpoints={}


### cnt分箱

In [9]:
cutlist=[(0.01,100),(0.02,50),(0.03,33),(0.05,20),(0.1,10)]

cntcol=list(filter(lambda x:"cnt" in x,df.columns))
cnt_split_expr={}
for col in cntcol:
    for cutset in cutlist:
        breakpoint_df=df[col].qcut([cutset[0]*i for i in range(1,cutset[1])],allow_duplicates=True,include_breaks=True) \
                        .struct.field('break_point') \
                        .unique() \
                        .sort()[:-1]
        if len(breakpoint_df)<20:
            ans=list(breakpoint_df.cast(pl.Int32))
            split_tab[col]=ans
            # print(list(cutdf["break_point"].unique_counts()))
            # print(ans)
            expr=interval2id(col,split_tab[col]).alias(col)
            breakpoints[col]=ans
            cnt_split_expr[col]=expr
            # cnt_split_expr.append(expr)
            break


### other

In [10]:
cutlist=[(0.001,1000),(0.005,200),(0.01,100),(0.02,50),(0.03,33),(0.05,20),(0.1,10)]

othercol=df.select(pl.col(pl.Float64)).select(pl.exclude("^.*cnt.*$")).columns
other_split_expr={}
for col in othercol:
    for cutset in cutlist:
        breakpoint_df=df[col].qcut([cutset[0]*i for i in range(1,cutset[1])],allow_duplicates=True,include_breaks=True) \
                        .struct.field('break_point') \
                        .unique() \
                        .sort()[:-1]
        if len(breakpoint_df)<50:
            ans=list(filter(lambda x:x is not None,list(breakpoint_df)))
            if ans[-1]<1:
                for i in range(len(ans)):
                    if ans[i]>0.99:
                        break
                ans=ans[:i+1]

            split_tab[col]=ans
            # print(list(cutdf["break_point"].unique_counts()))
            # print(ans)
            expr=interval2id(col,split_tab[col]).alias(col)
            breakpoints[col]=ans
            other_split_expr[col]=expr
            # cnt_split_expr.append(expr)
            break


In [11]:
exprlist=[
        pl.col("viptypes").map_dict(viptypes_map,default=0,return_dtype=pl.Int8),
        pl.col("category").map_dict(category_map,default=0,return_dtype=pl.Int8),
        pl.col("asrgender").map_dict(asrgender_map,default=0,return_dtype=pl.Int8),
        pl.col("respaytype").map_dict(respaytype_map,default=0,return_dtype=pl.Int8),
        pl.col("paytypematchflag").map_dict(paytypematchflag_map,default=0,return_dtype=pl.Int8),
        pl.col("selfFlag").map_dict(selfFlag_map,default=0,return_dtype=pl.Int8),
        pl.when(pl.col("usedaygap")<=14).then(pl.col("usedaygap")).otherwise(pl.lit(14)),
        interval2id("viptypeduetime",viptypeduetime_split).alias("viptypeduetime"),
        pl.when((pl.col("uid") != 0) & (pl.col("uid")!=-1))
            .then(pl.lit(1))
            .otherwise(pl.lit(0))
            .alias("login")
]
exprlist+=list(other_split_expr.values())+list(cnt_split_expr.values())


In [14]:
ld=pl.scan_csv("./part-00000-f8f702fe-fc48-48c9-9326-0b7ca76ecd77-c000.csv")
ld.select(
    pl.exclude(["initialid","requestid","resid","maxClickPosition"])
).with_columns(
    exprlist
).sink_csv("train.csv")

In [15]:
# ctr=["normslotmatchscore","normtrunkmatchscore","maxvipctr","maxallctr","slotQueryCtrSum","rawQueryCtrSum","maxAllSlotQueryCtr","maxAllRawQueryCtr"]
# rate=["metarating","doubanrating"]
# ratio=["metaclickscore","metaclickratio","metaplayratio","metaplaylength","realtimescore"]
# tmp.select(pl.col(pl.Float64)).describe()

In [16]:
import json
feature_config_path="./newconf/feature_config.json"
with open(feature_config_path,"r") as f:
    conf=json.load(f)

In [17]:
for idx,(col,val) in enumerate(split_tab.items()):
    conf[col]={
        "default_value": 0,
        "type": "bucketid",
        "embedding_dim": 16,
        "field": "other",
        "field_name": f"field{12+idx}",
        "hash_bucket": len(val)+1
    }

with open(feature_config_path,"w") as f:
    json.dump(conf,f,indent=2)

## 划分数据,转成tfrecord

In [18]:
from utils.tfrecords import pl2tf

2023-10-03 22:37:57.857996: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
import polars as pl

fd=pl.read_csv_batched("train.csv",batch_size=1048000)

In [20]:
for i in range(1000):
    try:
        subcsv=subdf=fd.next_batches(1)[0]
        subcsv=subcsv.sample(fraction=1, shuffle=True)
        train_size=int(len(subcsv)*0.7)
        val_size=int(len(subcsv)*0.15)
        test_size=len(subcsv)-val_size-train_size
        traincsv=subcsv.head(train_size)
        tmp=subcsv.tail(len(subcsv)-train_size)
        valcsv=tmp.head(val_size)
        testcsv=tmp.tail(test_size)
        traincsv.write_csv(f"./data/train/{i:02}.csv")
        valcsv.write_csv(f"./data/val/{i:02}.csv")
        testcsv.write_csv(f"./data/test/{i:02}.csv")
    except:
        break

In [16]:
df.head(1)

initialid,requestid,appid,uid,viptypes,asrgender,position,resid,category,respaytype,selfFlag,normslotmatchscore,trunkmatchscore,normtrunkmatchscore,maxvipctr,maxallctr,metarating,doubanrating,metaclickscore,metaclickratio,metaplayratio,metaplaylength,realtimescore,relativerealtime,popularityscore,queryvideopvcgmatchscore,paytypematchflag,viptypeuserpaycnt,viptypeuserpayfee,viptypeduetime,ipresusercnt,categoryusercnt,cpusercnt,respaycount,queryCtrConfidenceScore,slotCtrConfidenceScore,slotQueryCtrSum,rawQueryCtrSum,maxAllSlotQueryCtr,maxAllRawQueryCtr,requestcnt,clickcnt,playcnt,playdurationsum,usedaygap,payclickcnt,payplaycnt,payplaydurationsum,payfeesum,payfeeavg,payfeecnt,paysinglecnt,payvipcnt,finalClickFlag,pay_flag,maxClickPosition
str,str,i64,i64,str,str,i64,i64,str,str,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,i64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,f64,i64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,i64
"""1008dc65220748…","""1008dc65220748…",648470631775472640,2444780292,"""[]""","""female""",0,655832,"""音乐""","""PAY""",true,0.923841,7.050016,0.999986,0.0,0.4813,0.0,0.0,0.351177,0.09325,0.073761,860.336957,0.88898,53.0,0.796582,0.970575,false,0,0.0,0,0,0,0,42.0,0.0,1.0884,0.69,0.0,0.4813,0.0,0,0,0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,1,0,0


In [36]:
import glob
train_list=glob.glob("./data/train/*.csv")
tmp=pl.read_csv(train_list[0])
_pl2tf(tmp,'./data/train/04.tf')

In [55]:
class CSVDataSet():
    def __init__(self,filelist,record_defaults,column_names,drop_columns=[],n_readers=4,num_parallel_calls=4,buffer_size=10240):
        self.filename_dataset = tf.data.Dataset.list_files(train_list)
        self.n_readers=n_readers
        self.record_defaults=record_defaults
        self.column_names=column_names
        self.drop_columns=drop_columns
        self.buffer_size=buffer_size
        self.num_parallel_calls=num_parallel_calls

    def getds(self):
        dataset = self.filename_dataset.interleave(
            lambda filename: tf.data.TextLineDataset(filename).skip(1).shuffle(self.buffer_size),
            #并行数为5，默认一次从并行数中取出一条数据
            cycle_length = n_readers
        )
        dataset = dataset.map(self.parse_csv, num_parallel_calls=self.num_parallel_calls)
        return dataset

    def parse_csv(self,value):
        columns = tf.io.decode_csv(value, record_defaults=self.record_defaults)
        features = dict(zip(self.column_names, columns))
        for col in self.drop_columns :
            features.pop(col)
        # for col in self.sequental_columns:
        #     features[col]=tf.strings.to_number(tf.strings.split(features[col], sep='|').values, out_type=tf.dtypes.int64)
        ctr = features.pop('finalClickFlag')
        cvr = features.pop('pay_flag')
        ctr = tf.to_float(tf.reshape(ctr, [1]))
        cvr = tf.to_float(tf.reshape(cvr, [1]))
        return features, {'ctr': ctr, 'cvr': cvr}

record_defaults=[]
column_names=[]
drop_columns=["uid"]
train_list=glob.glob("./data/train/*.csv")

tmp=CSVDataSet(train_list,record_defaults,column_names,drop_columns)
ds=tmp.getds()


ValueError: in user code:

    File "/tmp/ipykernel_417572/257636309.py", line 21, in parse_csv  *
        columns = tf.io.decode_csv(value, record_defaults=self.record_defaults)

    ValueError: Attr 'OUT_TYPE' of 'DecodeCSV' Op passed list of length 0 less than minimum 1.


In [ ]:

def csv2dataset(pattern,n_readers=5,buffer_size=102400):
    filelist=glob.glob(patter)
    filename_dataset = tf.data.Dataset.list_files(train_list)
    dataset = filename_dataset.interleave(
        lambda filename: tf.data.TextLineDataset(filename).skip(1).shuffle(buffer_size),
        #并行数为5，默认一次从并行数中取出一条数据
        cycle_length = n_readers
    )
    dataset = dataset.map(parse_csv, num_parallel_calls=8)

In [38]:
filename_dataset = tf.data.Dataset.list_files(train_list)
n_readers = 5
dataset = filename_dataset.interleave(
    #skip()跳过第一行header
    lambda filename: tf.data.TextLineDataset(filename).skip(1).shuffle(shuffle_buffer_size),
    #并行数为5，默认一次从并行数中取出一条数据
    cycle_length = n_readers
)

for line in dataset.take(5):
    # print(line.numpy())

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
b'262701464776869888,48359979,0,1,17,\xe5\x84\xbf\xe7\xab\xa5,PAY,1,28,21,11,0,0,27,0,24,30,32,18,38,26,19,11,0,0,0,6,0,0,0,3,12,10,7,7,7,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1'
b'738426832184935424,2344032245,3,0,134,\xe5\x84\xbf\xe7\xab\xa5,PAY,1,11,33,26,0,0,0,0,35,33,35,25,42,0,39,14,1,1,32,11,0,15,12,24,12,10,7,7,7,6,12,12,12,42,1,12,12,41,32,37,1,0,1,0,0,1'
b'404959229649101824,1149818388,3,1,7,\xe5\x84\xbf\xe7\xab\xa5,PAY,1,2,7,6,0,12,0,0,34,31,34,17,41,4,33,11,1,0,0,6,0,0,0,17,7,9,21,28,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1'
b'738426832184935424,80864173,0,2,0,\xe9\x9f\xb3\xe4\xb9\x90,FREE,1,22,34,30,0,42,31,0,39,44,46,44,40,17,48,40,0,0,0,6,0,0,0,23,0,34,21,0,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1'
b'262701464776869888,2226801868,1,0,1,\xe5\x84\xbf\xe7\xab\xa5,FREE,1,28,39,30,22,34,33,0,4

In [90]:
ans=tf.io.decode_csv(b'404959229649101824,1149818388,',["1","1",""])

In [70]:
tf.strings.to_number(tf.strings.split(ans[2], sep='|'))

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1231., 1231.], dtype=float32)>